Imoprting postgresql database

In [ ]:
import psycopg2
print (psycopg2.__libpq_version__)

Establishing connection between database and python, also collecting information from database

In [ ]:
connection = psycopg2.connect(user = 'postgres',
                              password = 'itmm1315@VZU',
                              host = 'localhost',
                              port = '5432',
                              database = 'predictions')
cursor = connection.cursor()
cursor.execute('Select * from house_amsterdam')
result = cursor.fetchall()
print(result)
cursor.close()

Importing information from sql data using pandas 

In [ ]:
import pandas as pd
df = pd.read_sql_query("Select * from house_amsterdam", connection)
print(df.describe())

Plotting all of the data against each other using seaborn library to understand the relationship

In [ ]:
import seaborn as sns
print(df.columns)
print(df.info())
sns.pairplot(df)

Defining features and output

In [ ]:
x1 = df[['price', 'area', 'room', 'lon', 'lat']]
x2 = df[['index', 'address', 'zip']]

X = x1.drop("price", axis=True)
Y = df['price']
sns.pairplot(X)
print(Y.head())
print(Y.describe())
print(Y.isnull().sum())

EDA --Explatory data analysis

Dropping NaN and Filling with mean values in price column

In [ ]:
Y = pd.to_numeric(Y, errors="coerce")
Y = Y.fillna(Y.mean())
df["Y"] = Y

In [ ]:
print(X.corr())
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
sns.heatmap(X.corr(), annot=True, cmap="Blues")
plt.title('Correlation Heatmap for Features')

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
sns.boxplot(data = X, ax=ax)
plt.savefig('boxplot.jpg')

Scaling of features using standardscaler

Importing train test split from sklearn

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import numpy as np
from sklearn.model_selection import train_test_split
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
print('Length of X_test_scaled:', len(X_test_scaled))
print('Length of X_train_scaled:', len(X_train_scaled))
print('X_test:', X_test[:5])
print('\nX_test_scaled:', X_test_scaled)


Boxplot to identify outliers in X_train_scaled dataset

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
sns.boxplot(data = X_train_scaled, ax=ax)
plt.savefig('boxplot.jpg')

Boxplot to identify outliers in X_test_scaled dataset

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.boxplot(data = X_test_scaled, ax=ax)
plt.savefig('boxplot.jpg')

Fixing outliers using IQR method

In [ ]:
X_train_scaled_df = pd.DataFrame(
    X_train_scaled,
    columns=X.columns
)
X_test_scaled_df = pd.DataFrame(
    X_test_scaled,
    columns=X.columns
)
X_train_scaled_df_reset = X_train_scaled_df.reset_index(drop=True)
Y_train_reset = Y_train.reset_index(drop=True)
print(X_train_scaled_df_reset.shape, Y_train_reset.shape)
Q1 =X_train_scaled_df_reset.quantile(0.25)
Q3 = X_train_scaled_df_reset.quantile(0.75)
IQR = Q3-Q1
lower_limit =Q1 - 1.5*IQR
upper_limit = Q3 + 1.5*IQR
print('Lower Limit:', lower_limit)
print('Uower Limit:', upper_limit)
outliers = ((X_train_scaled_df_reset<lower_limit)|(X_train_scaled_df_reset>upper_limit)).any(axis =1)
print(outliers)
print("Number of outlier rows:", outliers.sum())
X_train_clean = X_train_scaled_df_reset[~outliers]  
outliers_aligned = outliers.reindex(Y_train_reset.index, fill_value=False)
Y_train_clean = Y_train_reset[~outliers_aligned]      


Building Linear Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression
print(X_train_clean.shape)
print(Y_train_clean.shape)
print(X_train_clean.head(), Y_train_clean.head())
linear_reg = LinearRegression().fit(X_train_clean, Y_train_clean)
Y_predict_LR = linear_reg.predict(X_test_scaled)
Y_predict_LR_train = linear_reg.predict(X_train_scaled)

Evaluation matrices for Linear Regression

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score, mean_squared_error
mse_LR = mean_squared_error(Y_test, Y_predict_LR)
mae_LR = mean_absolute_error(Y_test, Y_predict_LR)
r2_LR = r2_score(Y_test, Y_predict_LR)
r2_LR_train = r2_score(Y_train, Y_predict_LR_train)

Applying cross-validation for testing

In [ ]:
from sklearn.model_selection import cross_val_score, cross_validate
cv_results_LR = cross_validate(LinearRegression(), X_train_clean, Y_train_clean, 
                           cv=5, 
                           scoring=['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error'])
r2_LR_CV = cv_results_LR['test_r2'].mean().round(3)
mse_LR_CV =cv_results_LR['test_neg_mean_squared_error'].mean().round(3)
mae_LR_CV = cv_results_LR['test_neg_mean_absolute_error'].mean().round(3)

Plotting results

In [ ]:
min_val = min(Y_test.min(), Y_predict_LR.min())
max_val = max(Y_test.max(), Y_predict_LR.max())
plt.figure(figsize =(8, 5))
plt.scatter(Y_test, Y_predict_LR)
plt.plot([min_val, max_val], [min_val, max_val])
plt.text(min_val, max_val, f"R²_test = {r2_LR:3f}\nR²_train = {r2_LR_train:.3f}\nR²_CV = {r2_LR_CV:.3f}", 
         verticalalignment = 'top')
plt.xlabel('Actual Value')
plt.ylabel('Predicted Values')
plt.title('Linear Regression Model Predictions')
plt.show()

In [ ]:
residuals_LR = Y_test -Y_predict_LR
print(residuals_LR)
sns.displot(residuals_LR, kind='kde')
plt.title('Residual plot')

Transforming Price data (target) using log1p

In [ ]:
Y_train_clean_log_LR = np.log1p(Y_train_clean)
Y_train_log_LR = np.log1p(Y_train)
Y_test_log_LR = np.log1p(Y_test)
linear_reg = LinearRegression().fit(X_train_clean, Y_train_clean_log_LR)
Y_predict_LR_log = linear_reg.predict(X_test_scaled)
Y_predict_LR_log_train = linear_reg.predict(X_train_scaled)
min_val = min(Y_test_log_LR.min(), Y_predict_LR_log.min())
max_val = max(Y_test_log_LR.max(), Y_predict_LR_log.max())
mse_LR_log = mean_squared_error(Y_test_log_LR, Y_predict_LR_log)
mae_LR_log = mean_absolute_error(Y_test_log_LR, Y_predict_LR_log)
r2_LR_log = r2_score(Y_test_log_LR, Y_predict_LR_log)
r2_LR_log_train = r2_score(Y_train_log_LR, Y_predict_LR_log_train)

Applying cross-validation for testing on transformed data based linear regression model

In [ ]:
cv_results_LR_log = cross_validate(LinearRegression(), X_train_clean, Y_train_clean_log_LR, 
                           cv=5, 
                           scoring=['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error'])
r2_LR_CV_log = cv_results_LR_log['test_r2'].mean().round(3)
mse_LR_CV_log =cv_results_LR_log['test_neg_mean_squared_error'].mean().round(3)
mae_LR_CV_log = cv_results_LR_log['test_neg_mean_absolute_error'].mean().round(3)

Plotting results

In [ ]:
plt.figure(figsize =(8, 5))
plt.scatter(Y_test_log_LR, Y_predict_LR_log)
plt.plot([min_val, max_val], [min_val, max_val])
plt.text(min_val, max_val, f"R²_test = {r2_LR_log:3f}\nR²_train = {r2_LR_log_train:.3f}\nR²_CV = {r2_LR_CV_log:.3f}", 
         verticalalignment = 'top')
plt.xlabel('Actual Value')
plt.ylabel('Predicted Values')
plt.title('Linear Regression Model Predictions using log1p Transformation of Target')
plt.show()
residuals_LR_log = Y_test_log_LR -Y_predict_LR_log
sns.displot(residuals_LR_log, kind='kde')
plt.title('Residual')

Fitting Non-linear regression model (SVR)

In [ ]:
from sklearn.svm import SVR
svr_rbf =SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_NLR = svr_rbf.fit(X_train_clean, Y_train_clean)
Y_predict_SVR = np.asarray(svr_NLR.predict(X_test_scaled))
Y_predict_SVR_train = np.asarray(svr_NLR.predict(X_train_scaled))
mse_svr = mean_squared_error(Y_test, Y_predict_SVR)
mae_svr = mean_squared_error(Y_test, Y_predict_SVR)
r2_svr = r2_score(Y_test, Y_predict_SVR)
r2_svr_train = r2_score(Y_train, Y_predict_SVR_train)

Applying cross-validation for testing on SVR model

In [ ]:
cv_results_svr = cross_validate(svr_rbf, X_train_clean, Y_train_clean, 
                           cv=5, 
                           scoring=['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error'])
r2_svr_CV = cv_results_svr['test_r2'].mean().round(3)
mse_svr_CV =cv_results_svr['test_neg_mean_squared_error'].mean().round(3)
mae_svr_CV = cv_results_svr['test_neg_mean_absolute_error'].mean().round(3)

Plotting results for SVR

In [ ]:
plt.figure(figsize =(8, 5))
plt.scatter(Y_test, Y_predict_SVR)
min_val = min(Y_test.min(), Y_predict_SVR.min())
max_val = max(Y_test.max(), Y_predict_SVR.max())
plt.plot([min_val, max_val], [min_val, max_val])
plt.text(min_val, max_val, f"R²_test = {r2_svr:3f}\nR²_train = {r2_svr_train:.3f}\nR²_CV = {r2_svr_CV:.3f}", 
         verticalalignment = 'top')
plt.xlabel('Actual Value')
plt.ylabel('Predicted Values')
plt.title('SVR Model Predictions')
plt.show()

SVR with log1p transformation

In [ ]:
svr_NLR_log = svr_rbf.fit(X_train_clean, Y_train_clean_log_LR)
Y_predict_SVR_log = np.asarray(svr_NLR_log.predict(X_test_scaled))
Y_predict_SVR_log_train = np.asarray(svr_NLR_log.predict(X_train_scaled))
mse_svr_log = mean_squared_error(Y_test_log_LR, Y_predict_SVR_log)
mae_svr_log = mean_absolute_error(Y_test_log_LR, Y_predict_SVR_log)
r2_svr_log = r2_score(Y_test_log_LR, Y_predict_SVR_log)
r2_svr_log_train = r2_score(Y_train_log_LR, Y_predict_SVR_log_train)

Applying cross-validation for testing on log1p transformed data based SVR model

In [ ]:
cv_results_svr_log = cross_validate(svr_rbf, X_train_clean, Y_train_clean_log_LR, 
                           cv=5, 
                           scoring=['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error'])
r2_svr_log_CV = cv_results_svr_log['test_r2'].mean().round(3)
mse_svr_log_CV =cv_results_svr_log['test_neg_mean_squared_error'].mean().round(3)
mae_svr_log_CV = cv_results_svr_log['test_neg_mean_absolute_error'].mean().round(3)

Plotting results for SVR

In [ ]:
plt.figure(figsize =(8, 5))
plt.scatter(Y_test_log_LR, Y_predict_SVR_log)
min_val = min(Y_test_log_LR.min(), Y_predict_SVR_log.min())
max_val = max(Y_test_log_LR.max(), Y_predict_SVR_log.max())
plt.plot([min_val, max_val], [min_val, max_val])
plt.text(min_val, max_val, f"R²_test = {r2_svr_log:3f}\nR²_train = {r2_svr_log_train:.3f}\nR²_CV = {r2_svr_log_CV:.3f}", 
         verticalalignment = 'top')
plt.xlabel('Actual Value')
plt.ylabel('Predicted Values')
plt.title('SVR Model Predictions using log1p Transformation of Target')
plt.show()

Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf_reg = rf.fit(X_train_clean, Y_train_clean)
Y_predict_RF = rf_reg.predict(X_test_scaled)
Y_predict_RF_train = rf_reg.predict(X_train_scaled)
residuals_RF = Y_test - Y_predict_RF
mse_RF = mean_squared_error(Y_test, Y_predict_RF)
mae_RF = mean_absolute_error(Y_test, Y_predict_RF)
r2_RF = r2_score(Y_test, Y_predict_RF)
r2_RF_train = r2_score(Y_train, Y_predict_RF_train)

Applying cross-validation for testing RF model

In [ ]:
cv_results_RF = cross_validate(rf_reg, X_train_clean, Y_train_clean, 
                           cv=5, 
                           scoring=['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error'])
r2_RF_CV = cv_results_RF['test_r2'].mean().round(3)
mse_RF_CV =cv_results_RF['test_neg_mean_squared_error'].mean().round(3)
mae_RF_CV = cv_results_RF['test_neg_mean_absolute_error'].mean().round(3)

Plotting results for Random Forest

In [ ]:
plt.figure(figsize =(8, 5))
plt.scatter(Y_test, Y_predict_RF)
min_val = min(Y_test.min(), Y_predict_RF.min())
max_val = max(Y_test.max(), Y_predict_RF.max())
plt.plot([min_val, max_val], [min_val, max_val])
plt.text(min_val, max_val, f"R²_test = {r2_RF:3f}\nR²_train = {r2_RF_train:.3f}\nR²_CV = {r2_RF_CV:.3f}", 
         verticalalignment = 'top')
plt.xlabel('Actual Value')
plt.ylabel('Predicted Values')
plt.title('Random Forest Model Predictions')
plt.show()
sns.displot(residuals_RF, kind='kde')
plt.title('Residuals')

Now transforming Y_train and Y_test data using log1p for RF

In [ ]:
rf_reg_log = rf_reg.fit(X_train_clean, Y_train_clean_log_LR)
Y_predict_RF_log = rf_reg_log.predict(X_test_scaled)
Y_predict_RF_log_train = rf_reg_log.predict(X_train_scaled)
residuals_RF_log = Y_test - Y_predict_RF_log
mse_RF_log = mean_squared_error(Y_test, Y_predict_RF_log)
mae_RF_log = mean_absolute_error(Y_test, Y_predict_RF_log)
r2_RF_log = r2_score(Y_test_log_LR, Y_predict_RF_log)
r2_RF_log_train = r2_score(Y_train_log_LR, Y_predict_RF_log_train)

Applying cross validation

In [ ]:
cv_results_RF_log = cross_validate(rf_reg, X_train_clean, Y_train_clean_log_LR, 
                           cv=5, 
                           scoring=['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error'])
r2_RF_log_CV = cv_results_RF_log['test_r2'].mean().round(3)
mse_RF_log_CV =cv_results_LR['test_neg_mean_squared_error'].mean().round(3)
mae_RF_log_CV = cv_results_LR['test_neg_mean_absolute_error'].mean().round(3)

Plotting for Random Forest

In [ ]:
plt.figure(figsize =(8, 5))
plt.scatter(Y_test_log_LR, Y_predict_RF_log)
min_val = min(Y_test_log_LR.min(), Y_predict_RF_log.min())
max_val = max(Y_test_log_LR.max(), Y_predict_RF_log.max())
plt.plot([min_val, max_val], [min_val, max_val])
plt.text(min_val, max_val, f"R²_test = {r2_RF_log:3f}\nR²_train = {r2_RF_log_train:.3f}\nR²_CV = {r2_RF_log_CV:.3f}", 
         verticalalignment = 'top')
plt.xlabel('Actual Value')
plt.ylabel('Predicted Values')
plt.title('Random Forest Model Predictions using log1p Transformation of Target')
plt.show()
sns.displot(residuals_RF_log, kind='kde')
plt.title('Residuals')

XGBoost regression model 

In [ ]:
import xgboost as xgb
xgb_reg = xgb.XGBRegressor(n_estimators = 100, learning_rate = 0.1, max_depth = 6)
XGB_Reg = xgb_reg.fit(X_train_clean, Y_train_clean)
Y_predict_XGB = XGB_Reg.predict(X_test_scaled)
Y_predict_XGB_train = XGB_Reg.predict(X_train_scaled)
residuals_XGB = Y_test - Y_predict_XGB
mse_XGB = mean_squared_error(Y_test, Y_predict_XGB)
mae_XGB = mean_absolute_error(Y_test, Y_predict_XGB)
r2_XGB = r2_score(Y_test, Y_predict_XGB)
r2_XGB_train = r2_score(Y_train, Y_predict_XGB_train)

Applying cross-validation for testing on XGBoost model

In [ ]:
cv_results_XGB = cross_validate(xgb_reg, X_train_clean, Y_train_clean, 
                           cv=5, 
                           scoring=['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error'])
r2_XGB_CV = cv_results_XGB['test_r2'].mean().round(3)
mse_XGB_CV =cv_results_XGB['test_neg_mean_squared_error'].mean().round(3)
mae_XGB_CV = cv_results_XGB['test_neg_mean_absolute_error'].mean().round(3)

Plotting XGBoost results

In [ ]:
plt.figure(figsize =(8, 5))
plt.scatter(Y_test, Y_predict_XGB)
min_val = min(Y_test.min(), Y_predict_XGB.min())
max_val = max(Y_test.max(), Y_predict_XGB.max())
plt.plot([min_val, max_val], [min_val, max_val])
plt.text(min_val, max_val, f"R²_test = {r2_XGB:3f}\nR²_train = {r2_XGB_train:.3f}\nR²_CV = {r2_XGB_CV:.3f}", 
         verticalalignment = 'top')
plt.xlabel('Actual Value')
plt.ylabel('Predicted Values')
plt.title('XGBoost Model Predictions')
plt.show()
sns.displot(residuals_XGB, kind='kde')
plt.title('Residuals')

Now transforming Y_train and Y_test data using log1p for XGBoost

In [ ]:
xgb_reg = xgb.XGBRegressor(n_estimators = 100, learning_rate = 0.1, max_depth = 6)
XGB_Reg_log = xgb_reg.fit(X_train_clean, Y_train_clean_log_LR)
Y_predict_XGB_log = XGB_Reg_log.predict(X_test_scaled)
Y_predict_XGB_log_train = XGB_Reg_log.predict(X_train_scaled)
residuals_XGB = Y_test_log_LR - Y_predict_XGB_log
mse_XGB_log = mean_squared_error(Y_test_log_LR, Y_predict_XGB_log)
mae_XGB_log = mean_absolute_error(Y_test_log_LR, Y_predict_XGB_log)
r2_XGB_log = r2_score(Y_test_log_LR, Y_predict_XGB_log)
r2_XGB_log_train = r2_score(Y_train_log_LR, Y_predict_XGB_log_train)

Applying cross-validation for testing on log1p transformed data based XGBoost model

In [ ]:
cv_results_XGB_log = cross_validate(xgb_reg, X_train_clean, Y_train_clean_log_LR, 
                           cv=5, 
                           scoring=['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error'])
r2_XGB_CV_log = cv_results_XGB_log['test_r2'].mean().round(3)
mse_XGB_CV_log =cv_results_XGB_log['test_neg_mean_squared_error'].mean().round(3)
mae_XGB_CV_log = cv_results_XGB_log['test_neg_mean_absolute_error'].mean().round(3)
plt.figure(figsize =(8, 5))
plt.scatter(Y_test_log_LR, Y_predict_XGB_log)
min_val = min(Y_test_log_LR.min(), Y_predict_XGB_log.min())
max_val = max(Y_test_log_LR.max(), Y_predict_XGB_log.max())
plt.plot([min_val, max_val], [min_val, max_val])
plt.text(min_val, max_val, f"R²_test = {r2_XGB_log:3f}\nR²_train = {r2_XGB_log_train:.3f}\nR²_CV = {r2_XGB_CV_log:.3f}", 
         verticalalignment = 'top')
plt.xlabel('Actual Value')
plt.ylabel('Predicted Values')
plt.title('XGBoost Model Predictions using log1p Transformation of Target')
plt.show()
sns.displot(residuals_XGB, kind='kde')
plt.title('Residual')

Evaluating performance

In [ ]:
mse_values = {"Linear":mse_LR, "SVR":mse_svr, "Random Forest":mse_RF, "XGBoost":mse_XGB}
mse_values_trans = {"Linear with log transformation":mse_LR_log, "SVR with log1p transformation":mse_svr_log,
                    "Random Forest with log1p transformation":mse_RF_log,
                    "XGBoost with log1p transformation":mse_XGB_log}

mse_values_CV = {"Linear":mse_LR_CV, "SVR":mse_svr_CV, "Random Forest":mse_RF_CV, "XGBoost":mse_XGB_CV}
mse_values_trans_CV = {"Linear with log transformation":mse_LR_CV_log, 
                       "SVR with log1p transformation":mse_svr_log_CV,
                    "Random Forest with log1p transformation":mse_RF_log_CV,
                    "XGBoost with log1p transformation":mse_XGB_CV_log}

mae_values = {"Linear":mae_LR, "SVR":mae_svr, "Random Forest":mae_RF,"XGBoost":r2_XGB}
mae_values_trans = {"Linear with log transformation":mae_LR_log,
                    "SVR with log1p transformation":mae_svr_log,
                    "Random Forest with log1p transformation":mae_RF_log,
                    "XGBoost with log1p transformation":mae_XGB_log}

mae_values_CV = {"Linear":mae_LR_CV, "SVR":mae_svr_CV, "Random Forest":mae_RF_CV,"XGBoost":r2_XGB_CV}
mae_values_trans_CV = {"Linear with log transformation":mae_LR_CV_log,
                    "SVR with log1p transformation":mae_svr_log_CV,
                    "Random Forest with log1p transformation":mae_RF_log_CV,
                    "XGBoost with log1p transformation":mae_XGB_CV_log}

r2_values = {"Linear":r2_LR, "SVR":r2_svr, "Random Forest":r2_RF, "XGBoost":r2_XGB}
r2_values_train = {"Linear":r2_LR_train, "SVR":r2_svr_train, "Random Forest":r2_RF_train, "XGBoost":r2_XGB_train}
r2_values_trans = {"Linear with log transformation":r2_LR_log, 
             "SVR with log1p transformation":r2_svr_log, 
             "Random Forest with log1p transformation":r2_RF_log, 
             "XGBoost with log1p transformation":r2_XGB_log}
r2_values_trans_train = {"Linear with log transformation":r2_LR_log_train, 
             "SVR with log1p transformation":r2_svr_log_train, 
             "Random Forest with log1p transformation":r2_RF_log_train, 
             "XGBoost with log1p transformation":r2_XGB_log_train}
r2_values_CV = {"Linear":r2_LR_CV, "SVR":r2_svr_CV, "Random Forest":r2_RF_CV, "XGBoost":r2_XGB_CV}
r2_values_trans_CV = {"Linear with log transformation":r2_LR_CV_log, 
             "SVR with log1p transformation":r2_svr_log_CV, 
             "Random Forest with log1p transformation":r2_RF_log_CV, 
             "XGBoost with log1p transformation":r2_XGB_CV_log}

In [ ]:
max_r2 =max(r2_values.values())
max_r2_train =max(r2_values_train.values())
max_r2_CV =max(r2_values_CV.values())
max_r2_trans =max(r2_values_trans.values())
max_r2_trans_train =max(r2_values_trans_train.values())
max_r2_trans_CV =max(r2_values_trans_CV.values())
min_mse = min(mse_values.values())
min_mse_CV = min(mse_values_CV.values())
min_mse_trans = min(mse_values_trans.values())
min_mse_trans_CV = min(mse_values_trans_CV.values())
min_mae = min(mae_values.values())
min_mae_CV = min(mae_values_CV.values())
min_mae_trans = min(mae_values_trans.values())
min_mae_trans_CV = min(mae_values_trans_CV.values())
Best_model = max(r2_values, key=r2_values.get)
Best_model_CV = max(r2_values_CV, key=r2_values_CV.get)
Best_model_trans = max(r2_values_trans, key=r2_values_trans.get)
Best_model_trans_CV = max(r2_values_trans_CV, key=r2_values_trans_CV.get)

print('For non-transformed targets, best model is', Best_model, 'with R²_test-', max_r2, 'R²_train-',r2_values_train[Best_model], 
      'MSE', mse_values[Best_model], 'and MAE', mae_values[Best_model])
print('For transformed targets, best model is', Best_model_trans,'with R²_test-', max_r2_trans, 'R²_train-',
      r2_values_trans_train[Best_model_trans], 'MSE', mse_values_trans[Best_model_trans], 'and MAE', mae_values_trans[Best_model_trans])
print('For non-transformed targets, and cross-validation testing best model is', Best_model_CV, 'and R²_CV is', r2_values_CV[Best_model_CV])
print('For transformed targets, and cross validation testing best model is', Best_model_trans_CV,'and R²_CV is', r2_values_trans_CV[Best_model_trans_CV])
if max_r2>=max_r2_trans:
    print('Best Model is:', Best_model)
else:
    print('Best Model is:', Best_model_trans)

if max_r2_CV>=max_r2_trans_CV:
    print('Best Model for cross validation testing is:', Best_model_CV)
else:
    print('Best Model for cross validation testing is:', Best_model_trans_CV)